# RL Pipeline Notebook
This notebook runs the analysis pipeline using the translated Python modules.

In [ ]:
from importlib import import_module
setup = import_module('RL_Maggie.python.00_setup')
print('LOCAL_DATA_DIR:', setup.LOCAL_DATA_DIR)
print('STRESS_DATA_DIR:', setup.STRESS_DATA_DIR)


In [ ]:
# Optional: copy raw files from shared drive if available
# copy_mod = import_module('RL_Maggie.python.01_copy_raw_files')
# copy_mod.copy_files_from_rl(setup.RAW_SHARED_DIR, setup.LOCAL_DATA_DIR)


## Trial-Level Data

In [ ]:
fibro_mod = import_module('RL_Maggie.python.02_trial_etl_fibro')
stress_mod = import_module('RL_Maggie.python.03_trial_etl_stress')
res_fibro = fibro_mod.get_trial_data_healthy_fibro()
res_stress = stress_mod.get_trial_data_healthy_stress()


In [ ]:
merge_mod = import_module('RL_Maggie.python.04_trial_merge_clean')
full_trial_data = merge_mod.full_trial_data
full_trial_data_learners = merge_mod.full_trial_data_learners
all_non_learners = merge_mod.all_non_learners
print('Trial rows:', len(full_trial_data))


In [ ]:
analysis_mod = import_module('RL_Maggie.python.05_trial_analysis')


## Estimation-Level Data

In [ ]:
est_etl = import_module('RL_Maggie.python.06_estimation_etl')
full_estimation_data = est_etl.full_estimation_data
full_estimation_data_clean = est_etl.full_estimation_data_clean
print('Estimation rows:', len(full_estimation_data))


In [ ]:
import_module('RL_Maggie.python.07_estimation_analysis')


## Questionnaire Analysis

In [ ]:
try:
    import_module('RL_Maggie.python.08_questionnaire_analysis')
except FileNotFoundError as e:
    print('Questionnaire file missing:', e)
